
# Camada Prata — Dados Refinados

A camada Prata marca o início do processo de transformação dos dados brutos armazenados na Bronze. Nessa etapa, foram criadas novas tabelas derivadas da fonte original, contendo apenas as colunas relevantes para as análises posteriores. As colunas consideradas desnecessárias ou que não contribuiriam para o modelo dimensional foram removidas, resultando em estruturas mais limpas, organizadas e adequadas para o processamento analítico.

Essa preparação inicial garante que somente os atributos essenciais avancem para as próximas fases, reduzindo complexidade, melhorando a qualidade da informação e estabelecendo uma base consistente para futuras padronizações, tratamento de valores nulos, validações e construção do modelo Snowflake.

In [0]:
USE CATALOG mvp

In [0]:
USE SCHEMA silver

In [0]:
CREATE TABLE atores_diretores
SELECT person_id, id, name, character AS perso, role
FROM mvp.bronze.credits

In [0]:
ALTER TABLE atores_diretores
ALTER COLUMN person_id COMMENT 'Identificador único de cada pessoa na tabela.';

ALTER TABLE atores_diretores
ALTER COLUMN id COMMENT 'Identifica os títulos de filmes ou séries.';

ALTER TABLE atores_diretores
ALTER COLUMN name COMMENT 'Nome da pessoa relacionada ao título.';

ALTER TABLE atores_diretores
ALTER COLUMN perso COMMENT 'Função ou papel desempenhado pela pessoa no título.';

ALTER TABLE atores_diretores
ALTER COLUMN role COMMENT 'Posição ou atividade desempenhada pela pessoa no contexto do título (exemplo: ator, diretor).';


In [0]:
%python
from pyspark.sql import functions as F

df_bronze = spark.table("mvp.bronze.titles")

df_silver = (
    df_bronze
    # Tipos corrigidos
    .withColumn("release_year", F.regexp_replace("release_year", r"\.0$", "").cast("int"))
    .withColumn("runtime", F.regexp_replace("runtime", r"\.0$", "").cast("int"))
    .withColumn("seasons", F.regexp_replace("seasons", r"\.0$", "").cast("int"))
    .withColumn("imdb_score", F.col("imdb_score").cast("double"))
    .withColumn("tmdb_score", F.col("tmdb_score").cast("double"))
    # votos como inteiro real
    .withColumn("imdb_votes", F.round(F.col("imdb_votes").cast("double"), 0).cast("int"))
    # manter apenas colunas úteis
    .select(
        "id",
        "title",
        F.col("type").alias("category"),
        "release_year",
        "runtime",
        "seasons",
        "genres",
        "production_countries",
        "imdb_score",
        "imdb_votes",
        "tmdb_score"
    )
)

df_silver.write.format("delta").saveAsTable("mvp.silver.titles")



Nesta etapa, os dados da tabela mvp.bronze.titles foram transformados para a camada silver, com foco em padronização de tipos e seleção de atributos relevantes. Foram realizadas correções de tipos numéricos, convertendo campos originalmente tratados como string (como release_year, runtime e seasons) para inteiros, além da conversão das pontuações do IMDb e TMDb para tipo double. O campo imdb_votes foi ajustado para inteiro, garantindo consistência para análises quantitativas. Por fim, foram mantidas apenas as colunas consideradas essenciais ao modelo analítico, incluindo a renomeação do campo type para category, resultando em um conjunto de dados limpo, tipado e pronto para uso nas etapas analíticas e de modelagem dimensional.

In [0]:
SELECT * FROM mvp.silver.titles LIMIT 20;


In [0]:
ALTER TABLE titles
ALTER COLUMN id COMMENT 'Identificador único do título, usado para distinção e referência no banco de dados.';

ALTER TABLE titles
ALTER COLUMN title COMMENT 'Nome do título da mídia, como filmes e séries.';

ALTER TABLE titles
ALTER COLUMN category COMMENT 'Categoria da mídia, especificando se é um filme ou série.';

ALTER TABLE titles
ALTER COLUMN release_year COMMENT 'Ano em que o título foi lançado ou disponibilizado ao público.';

ALTER TABLE titles
ALTER COLUMN runtime COMMENT 'Duração total do título em minutos, indicando a extensão do filme ou episódio.';

ALTER TABLE titles
ALTER COLUMN genres COMMENT 'Lista de gêneros aos quais o título pertence, fornecendo uma ideia do estilo e temática.';

ALTER TABLE titles
ALTER COLUMN production_countries COMMENT 'Países onde o título foi produzido, fornecendo contexto sobre origem e local de produção.';

ALTER TABLE titles
ALTER COLUMN seasons COMMENT 'Número de temporadas que uma série possui; filmes apresentam este campo como NULL.';

ALTER TABLE titles
ALTER COLUMN imdb_score COMMENT 'Classificação do título no IMDb, variando de 0 a 10, refletindo a opinião do público.';

ALTER TABLE titles
ALTER COLUMN tmdb_score COMMENT 'Classificação do título no TMDb (The Movie Database), variando de 0 a 10.';


In [0]:
ALTER TABLE titles
ALTER COLUMN imdb_votes COMMENT 'Número de votos recebidos pelo título no IMDb, refletindo a participação e a opinião do público.';

In [0]:
DESCRIBE TABLE titles;


In [0]:
DESCRIBE TABLE atores_diretores

In [0]:
SELECT *
FROM titles
WHERE 
    id IS NULL
    AND title IS NULL
    AND category IS NULL
    AND release_year IS NULL  
    AND runtime IS NULL
    AND genres IS NULL
    AND production_countries IS NULL
    AND seasons IS NULL
    AND tmdb_score IS NULL
    AND imdb_score IS NULL
 


In [0]:
SELECT *
FROM titles
WHERE id IS NULL;


In [0]:
SELECT *
FROM titles
WHERE title IS NULL OR title = '';


In [0]:
SELECT *
FROM titles
WHERE runtime < 0;


In [0]:
SELECT *
FROM titles
WHERE release_year < 1888  -- 1º filme da história
   OR release_year > year(current_date());


In [0]:
SELECT *
FROM titles
WHERE tmdb_score < 0 OR tmdb_score > 10
   OR imdb_score < 0 OR imdb_score > 10;

In [0]:
SELECT id, COUNT(*) AS repeticoes
FROM titles
GROUP BY id
HAVING COUNT(*) > 1;


In [0]:
CREATE OR REPLACE TABLE titulo_genero AS
SELECT
    id AS id_titulo,
    genero_explodido AS genres
FROM titles
LATERAL VIEW OUTER explode(
    from_json(
        regexp_replace(genres, "'", '"'),
        'array<string>'
    )
) AS genero_explodido;



In [0]:
SELECT *
FROM titulo_genero
LIMIT 12

In [0]:
SELECT COUNT(*) AS qtd_nulls
FROM titulo_genero
WHERE genres IS NULL;


In [0]:
SELECT id, genres
FROM titles
WHERE genres IS NULL 
   OR genres = '[]' 
   OR genres = ''
   OR genres = '[""]';


In [0]:
CREATE OR REPLACE TABLE mvp.silver.titulo_pais AS
SELECT
    id,
    country
FROM (
    SELECT
        id,
        from_json(
            regexp_replace(production_countries, "'", '"'),
            'array<string>'
        ) AS countries
    FROM mvp.silver.titles
)
LATERAL VIEW explode(countries) AS country;



In [0]:
SELECT *
FROM mvp.silver.titulo_pais
LIMIT 12


# Resumo da Análise de Dados e Tratamento Estrutural (Camada Prata)

Após a aplicação das regras de validação, limpeza e transformação, foi realizada uma análise completa da qualidade dos dados, contemplando valores nulos, consistência dos tipos, integridade estrutural e comportamento das listas presentes no conjunto de títulos. As etapas executadas garantiram que o dataset fosse normalizado e preparado para a modelagem dimensional.

### Coluna seasons
Os valores nulos representam títulos que não são classificados como SHOW na coluna category.
Portanto, apenas séries possuem número de temporadas; filmes mantêm o campo como NULL.

### Coluna imdb_score
Valores nulos indicam títulos que não estão presentes no catálogo de avaliações do IMDb ou que ainda não receberam avaliação na plataforma no momento da coleta.

### Coluna tmdb_score
Valores NULL indicam que a pontuação não estava disponível na fonte TMDB ou que não pôde ser coletada durante o processo de ingestão, refletindo ausência de informação na origem.

### Colunas genres e production_countries
Embora não apresentem valores nulos, essas colunas incluem listas vazias quando não há informação disponível.
Durante a etapa Prata, esses campos passaram por:

- Conversão de strings para arrays válidos
- Explosão das listas (flatten)
- Criação das tabelas normalizadas:

    - titulo_genero

    - titulo_pais

Esse processo garantiu que os dados fossem estruturados corretamente para suportar a modelagem em estrela/snowflake na camada Ouro.

**Integridade Geral do Dataset**

Não foram identificados registros duplicados, valores impossíveis ou linhas totalmente inválidas.
As tabelas resultantes apresentam coerência estrutural e semântica, preservando fielmente as informações da fonte

**Conclusão**

A qualidade dos dados foi avaliada como adequada e consistente.
Com as transformações aplicadas, incluindo limpeza, padronização, normalização das listas e criação das tabelas intermediárias. a camada Prata está concluída, deixando o dataset totalmente preparado para as próximas etapas do projeto.